# Ingestion of Expert profile data to vector db and retrieval

This notebook extracts a json with expert profile information, transforms it into a document format, and then stores them into a vector database for subsequent processing. 

This is an additional index created alongside the watsonx docs rag vector index created in the **Process and Ingest into vector DB** notebook. 

The indexed documents are further used in the **Create and Deploy QnA Python Function** notebook which creates QnA RAG function, deploys it on watsonx.ai and can additionally retrieve the expert profile information in case the LLM has no answer to a specific question.

The process involves extracting content from the expert profiles file, segmenting the data, converting it into a document format, and finally indexing the content into a vector database. 

**Note:** A sample profile data is shipped with the accelerator. Alteratively you can index your own expert profile data instead.

The accelerator currently supports Elasticsearch and Milvus vector databases. The ingestion process uses vector embeddings to enhance data storage and retrieval within either Elasticsearch or Milvus vector databases, ensuring both efficiency and effectiveness.

* Establishing a connection to the chosen vector database (Elasticsearch or Milvus) and loading input data from processed documents.
* Generating unique IDs for documents.
* Inserting the documents with embeddings into Elasticsearch or Milvus using these generated IDs, with progress monitoring provided by a progress bar.

## Contents
* [Pre-Requisite Libraries and Dependencies](#setup)
* [Import Dependencies](#import)
* [Extract data from input file](#input)
* [Connect to Vector Database](#connect)
* [Index Documents using langchains vectorstore](#insert)
* [Search and Retrieve using Vectorstore and Query templates](#search)

<a id="setup"></a>
### Pre-Requisite Libraries and Dependencies
Download and import mandatory libraries and dependencies. 

Note : Some of the versions of the libraries may throw warnings after installation. These library versions are crucial for successful execution of the accelerator. Please ignore the warning/error and proceed with your execution. 

In [ ]:
!pip install elasticsearch | tail -n 1
!pip install langchain | tail -n 1
!pip install ibm_watsonx_ai==1.1.6 | tail -n 1
!pip install langchain_elasticsearch | tail -n 1
!pip install langchain_milvus==0.1.5 | tail -n 1

Restart the kernel after performing the pip install if the below cell fails to import all the libraries.

In [ ]:
from langchain.schema import Document
from ibm_watsonx_ai import APIClient,Credentials
import os,re
from tqdm import tqdm
import json
from ibm_watsonx_ai import __version__
import warnings
import hashlib
from ibm_watsonx_ai.foundation_models import Embeddings
from elasticsearch import Elasticsearch, helpers
from pymilvus import(Milvus,IndexType,Status,connections,FieldSchema,DataType,Collection,CollectionSchema,utility)
warnings.filterwarnings("ignore")

#### Get Environment variables

In [ ]:
project_id = os.environ['PROJECT_ID']
# Environment and host url
hostname = os.environ['RUNTIME_ENV_APSX_URL']

if hostname.endswith("cloud.ibm.com") == True:
    environment = "cloud"
else:
    environment = "on-prem"
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()

<a id="import"></a>
### Import Parameter Sets, credentials and Helper functions script.

#### Parameter sets import
Below cells imports parameter sets values, sets the watsonx.ai credentials and imports the helper functions script.

In [ ]:
parameter_sets = ["RAG_parameter_set","RAG_advanced_parameter_set"]
parameters={}
for parameter_set in parameter_sets:
    parameter_vals= wslib.assets.get_asset(parameter_set, "parameter_set", raw=True)
    if 'entity' in parameter_vals and 'parameter_set' in parameter_vals['entity'] and 'parameters' in parameter_vals['entity']['parameter_set']:
        parameter_vals = parameter_vals['entity']
    params = {param['name']: param['value'] for param in parameter_vals['parameter_set']['parameters']}
    parameters.update(params)

#### Setup watsonx.ai credentials 

In [ ]:
ibm_api_key=parameters['watsonx_ai_api_key']

if environment == "cloud":
    WML_SERVICE_URL = f"https://{hostname.split('.')[1]}.ml.cloud.ibm.com" if hostname.split('.')[0].endswith("api") and hostname.split('.')[2] == "dataplatform" else "https://us-south.ml.cloud.ibm.com"
    wml_credentials = Credentials(api_key=parameters['watsonx_ai_api_key'], url=WML_SERVICE_URL)
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials=Credentials(token=os.environ['USER_ACCESS_TOKEN'],url=hostname,instance_id='openshift')

#### RAG helper functions script import

In [ ]:
try:
    filename = 'rag_helper_functions.py'
    wslib.download_file(filename)
    import rag_helper_functions
    print("rag_helper_functions imported from the project assets")
except NameError as e:
    print(str(e))
    print("If running watsonx.ai aaS on IBM Cloud, check that the first cell in the notebook contains a project token. If not, select the vertical ellipsis button from the notebook toolbar and `insert project token`. Also check that you have specified your ibm_api_key in the second code cell of the notebook")


#### Setup the watsonx.ai client
Below cell uses the watson machine learning credentials to create an API client to interact with the project and deployment space.

In [ ]:
client = APIClient(wml_credentials)
client.set.default_project(project_id=project_id)

#### Import Expert Profiles

Change the `expert_profiles_document` parameter if you wish to use your own Expert Data to Ingest into the vector database

In [ ]:
# Get the ingestion document for indexing - Breakpoint for folks indexing their own expert profile data

filename = parameters['expert_profiles_document']
wslib.download_file(filename)
with open(filename) as f:
    expert_profiles = json.load(f)

<a id="input"></a>
### Extract the json document and convert the data into Langchain format for indexing
The cell below prepares documents for insertion into a vector database. 

In [ ]:
def convert_json_to_documents(json_data):
    documents = []
    for item in json_data:
        
        text = item.get("text", "")   
        # Extract the text with the profile information & rest is put into metadata
        metadata = {key: value for key, value in item.items() if key != "text"}  
        documents.append(Document(page_content=text, metadata=metadata))
    return documents

documents = convert_json_to_documents(expert_profiles)

In [ ]:
# Creating a hash lib 

content=[]
metadata = []
for doc in documents:
        
    metadata.append({
            
        "source" : doc.metadata['datainfo']['source'],
        "entry_number": doc.metadata['datainfo']['entry_number'],
        "name" : doc.metadata['datainfo']['name'],
        "email" : doc.metadata['datainfo']['email'],
        "domain" : doc.metadata['datainfo']['domain'],
        "document_id": doc.metadata['document_id']
        
        })
    
    content.append("Document Content: " + doc.page_content)
    
profile_documents = [Document(page_content=text, metadata=metadata) for text, metadata in zip(content, metadata)]

id_list=[]
for doc in profile_documents:
    id_list.append(hashlib.sha256(doc.page_content.encode()).hexdigest())  
print(len(id_list) - len(set(id_list)),"duplicate documents found.")

<a id="connect"></a>
## Connecting to Elastic Search/Milvus

The notebook, by default, will look for a connection asset in the project named milvus_connect or elasticsearch_connect. You can set this up by following the instructions in the project readme. This code checks if a specified connection exists in the project. If found, it retrieves the connection details and identifies the connection type. Depending on the connection type, it establishes a connection to the appropriate database. If the connection is not found, it raises an error indicating the absence of the specified connection in the project.

In [ ]:
connection_name=parameters["connection_asset"]

if(next((conn for conn in wslib.list_connections() if conn['name'] == connection_name), None)):
    print(connection_name, "Connection found in the project")
    db_connection = wslib.get_connection(connection_name)
    
    connection_datatypesource_id=client.connections.get_details(db_connection['.']['asset_id'])['entity']['datasource_type']
    connection_type = client.connections.get_datasource_type_details_by_id(connection_datatypesource_id)['entity']['name']
    
    print("Successfully retrieved the connection details")
    print("Connection type is identified as:",connection_type)
    
    regex = r'^[A-Za-z_]+[A-Za-z0-9_]*$' if connection_type == 'milvus' else r'^[a-z0-9$@=;!^(){}\[\]]+[a-z0-9$@=;!^(){}\[\]+-_.]*$'

    try:
        if not re.match(regex, parameters['expert_profiles_index']):
            raise ValueError(parameters['expert_profiles_index']," name can only contain letters, numbers, and underscores.")

        if connection_type=="elasticsearch":
            es_client=rag_helper_functions.create_elastic_client(db_connection)
        elif connection_type=="milvus":
            db_connection['database']=db_connection['database'] if 'database' in db_connection else 'default'

            connection_params = {
                'database': db_connection['database'],
                'host': db_connection['host'],
                'port': db_connection['port'],
                'user': db_connection['username'],
                'password': db_connection['password'],
                'secure': True
            }
            if 'ssl_certificate' in db_connection :
                ssl_certificate_content = db_connection.get('ssl_certificate') if db_connection.get('ssl_certificate') else ""
                cert_file_path = 'milvus_conn.cert'
                with open(cert_file_path, 'w') as file:
                    file.write(ssl_certificate_content)
                connection_params['server_pem_path'] = cert_file_path
            milvus_client = connections.connect(**connection_params)
            print("Successfully connected to milvus database")

    except ValueError as e:
        print(f'Error: Invalid index names provided: {e}. Please provide valid index names to proceed!!') 
else:
    db_connection=""
    raise ValueError(f"No connection named {connection_name} found in the project.")

<a id="insert"></a>
### Create vector database for Expert Profiles and index using Langchain vector store

Below code utilizes Langchains vector store extension to store documents.
The code sets up a vector store based on the specified connection type, either "elasticsearch" or "milvus".

* If connection_type is `"elasticsearch"`, it imports `ElasticsearchStore` from the `langchain_elasticsearch` library and initializes it with an Elasticsearch client and specified parameters with the given model ID. The Elastic Search vector store is then created using the `model_id`, connection parameters and index settings
* If connection_type is `"milvus"`, the code imports `langchain_milvus` and configures credentials using an API key and service URL. It initializes the embedding model specified in the parameters to generate embeddings, and sets up index parameters with specific metrics and configurations. The Milvus vector store is then created using the embedding function, connection details, and index settings. For cloud environments, it uses the `ibm_watsonx_ai` library to initialize the embedding function with the required API key, URL, and project ID. On Cloud Pak for Data software, it downloads the embedding model from HuggingFace via the `HuggingFaceEmbeddings` class to generate embeddings for the chunked documents.

Firstly, it creates a connection to the vector database and defines how documents will be retrieved later.
Then, it defines a function to add these documents to the vector store. This function takes the documents and additional parameters for efficient processing, such as splitting the documents into smaller chunks and setting a timeout for requests.

Overall, this code efficiently adds a list of documents to a vector store, thereby making them searchable

In [ ]:
if connection_type=="elasticsearch":
    from langchain_elasticsearch import ElasticsearchStore
    vector_store=ElasticsearchStore(
                    es_connection=es_client,
                    index_name=parameters["expert_profiles_index"],
                    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=parameters['elastic_search_model_id'])
                    )
    print("Elastic Search Vector Store Created with",parameters['elastic_search_model_id'])
elif connection_type=="milvus":
    from langchain_milvus import Milvus
    
    milvus_credentials={'database': db_connection['database'],'password':db_connection['password'] ,'port': db_connection['port'] ,'host': db_connection['host'],"secure": True,'user': db_connection['username']}
    print("using the model",parameters['embedding_model_id'], "to create embeddings")
    if environment=="cloud":
        credentials=Credentials(
            api_key = parameters['watsonx_ai_api_key'],
            url =WML_SERVICE_URL)
        embedding = Embeddings(
          model_id=parameters['embedding_model_id'],
          credentials=credentials,
          project_id=project_id,
          verify=True
        )
    elif environment=="on-prem":
        print("Downloading the embedding model from HuggingFace")
        from langchain_community.embeddings import HuggingFaceEmbeddings
        embedding=HuggingFaceEmbeddings(model_name=parameters['embedding_model_id'])
        
    index_params = {"index_type": "HNSW","metric_type": "L2",  "params": { "M": 16,"efConstruction": 200,"efSearch": 16 }}

    vector_store = Milvus(
        embedding_function=embedding,
        connection_args=milvus_credentials,
        index_params=index_params,
        primary_field='id',
        collection_name=parameters["expert_profiles_index"] 
         
    )
    print("Milvus Vector Store Created")



Below code defines a function to generates a list of unique IDs for each document by hashing their page_content. The code sets a chunk size for batch processing. It iterates over the documents in chunks, inserting each chunk into the vector store with corresponding IDs. The progress bar is updated to reflect the number of documents processed.


In [ ]:
def generate_hash(content):
    return hashlib.sha256(content.encode()).hexdigest()

In [ ]:
with tqdm(total=len(profile_documents), desc="Inserting Documents", unit="docs") as pbar:
    try:
        for i in range(0, len(profile_documents), 1):
            chunk = profile_documents[i:i + 1]
            id_chunk = [generate_hash(doc.page_content) for doc in chunk]
            vector_store.add_documents(chunk, ids=id_chunk)
            pbar.update(len(chunk))
        print("Documents are inserted into vector database")
    except Exception as e:
        print(f"An error occurred: {e}")

Above cell is a synchronous call & may take time to complete based on the size of the documents. You can proceed to **Create and Deploy Q&A Python Function** notebook to create and deploy the RAG python function without waiting for the previous cell to complete.

<a id="search"></a>
### Querying the vector index/collection for the expert profiles

The following sections of the notebook are designed to test a sample Question and Answer (QnA) interaction on the vector store. The subsequent cell in the notebook executes this test and provides a response that includes several key pieces of information.

In [ ]:
question ="What are risks in investment banking?"

In case of **Elastic Search**, The following section of the notebook is designed to test a sample Question and Answer (QnA) interaction using sample template of ELSER model, assuming it is utilized. This response comprises of: 

* `Document ID` : A unique identifier for the document within the database or index.
* `Document Context` : Expert Profile data content or text from the document that is relevant to the queried question.
* `Relevance Score` : A numerical value indicating the relevance or confidence level of the answer provided by the model.

This setup allows for a practical demonstration of the model's capabilities in retrieving and presenting information in response to a specific query. There are 2 ways to perform this step, depending on the `elastic_search_template_file` parameter provided in the parameter set by the user.

* **ELSER**: An ELSER exclusive search query is invoked.
* **ELSER + BM25**: A hybrid search query that is a combination of a tradition vector search and ELSER is invoked.


In case of **Milvus**, below code initializes a Milvus collection using a specified collection name and loads it into memory. It then generates embeddings for a given question using above specified embedding model. The code performs a vector search in the collection based on these embeddings, and retrieves the top closest result.


In [ ]:
############################ ELASTIC #################################

if connection_type=="elasticsearch":
    wslib.download_file(parameters['elastic_search_template_file'])
    with open(parameters['elastic_search_template_file']) as f:
        es_query = json.load(f)

    print("updating tokens and query fields")

    if "BM25" in parameters['elastic_search_template_file']:
        es_query['bool']['should'][0]['match']['description']['query'] = question
        es_query['bool']['should'][1]['text_expansion']['vector.tokens']['model_id'] = parameters['elastic_search_model_id']
        es_query['bool']['should'][1]['text_expansion']['vector.tokens']['model_text'] = question
        print("ELSER + BM25 Lexical search query : ")
    else:
        es_query['text_expansion']['vector.tokens']['model_id'] = parameters['elastic_search_model_id']
        es_query['text_expansion']['vector.tokens']['model_text'] = question
        print("ELSER search query : ")

    print(es_query)


    try:
        response = es_client.search(
            index=parameters["expert_profiles_index"], 
            size=1,
            query=es_query
        )
        print("\nResponse:")
        for hit in response['hits']['hits']:
            score = hit['_score']
            source = hit['_source']
            doc_id = source['metadata']['document_id']
            page_content = source['text']
            

            print(f"\nRelevance Score  : {score}\nDocument ID : {doc_id}\n\n{page_content}")

    
    except Exception as e:
            print("An error occurred while querying elastic search, please retry after sometime:", e)

############################ MILVUS #################################

elif connection_type=="milvus":
    if environment=="cloud":
        credentials=Credentials(
            api_key = parameters['watsonx_ai_api_key'],
            url =WML_SERVICE_URL)
        embedding = Embeddings(
          model_id=parameters['embedding_model_id'],
          credentials=credentials,
          project_id=project_id,
          verify=True
        )
    elif environment=="on-prem":
        print("Downloading the embedding model from HuggingFace")
        from langchain_community.embeddings import HuggingFaceEmbeddings
        embedding=HuggingFaceEmbeddings(model_name=parameters['embedding_model_id'])

    search_params = {"metric_type": "L2", "params": {"ef": 10, "range_filter": 0.1, "radius": 0.42}}
    
    query_embeddings = embedding.embed_documents(texts=[question])

    result = vector_store.similarity_search_with_score_by_vector(embedding.embed_query(question), k=1, param = search_params)
    try: 
        for res in result:
            print(res)
            print(f"\nRelevance Score  : {res[1]}\n\nDocument :-->\n\n{res[0]}")
            print('-----------')
    except Exception as e: 
        print("ERROR: No relevant information found. Please check all the parameters and try again.", e)


Proceed to the **Create and Deploy QnA Python Function** notebook to create and deploy the RAG python function.


**Sample Materials, provided under license.** <br>
**Licensed Materials - Property of IBM.** <br>
**© Copyright IBM Corp. 2024. All Rights Reserved.** <br>
**US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.** <br>